In [7]:
pip install selenium webdriver-manager pandas

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd

# 트위터 계정 정보
username = "YOUR_USERNAME"
email_or_phone = "YOUR_EMAIL_OR_PHONE"
password = "YOUR_PASSWORD"

# 검색 URL (latest로 강제 세팅)
search_url = "https://x.com/search?q=crypto%20regulation%20(bitcoin%20OR%20btc)%20-retweet%20-retweets%20-giveaway%20-free%20-win%20-promotion%20-promo%20-%23ad%20-%23sponsored%20lang%3Aen%20until%3A2025-03-06%20since%3A2025-02-06%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live"

# 셀레니움 설정
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

# 로그인 페이지 접속
driver.get("https://x.com/login")
time.sleep(3)

# 로그인 단계1 - 아이디/이메일 입력
email_input = wait.until(EC.presence_of_element_located((By.NAME, 'text')))
email_input.send_keys(email_or_phone)
email_input.send_keys(Keys.ENTER)
time.sleep(3)

# 사용자명 확인 창이 나오면 처리
try:
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.ENTER)
    time.sleep(3)
except:
    pass  # 사용자명 확인 창이 안나오면 패스

# 로그인 단계2 - 비밀번호 입력
password_input = wait.until(EC.presence_of_element_located((By.NAME, 'password')))
password_input.send_keys(password)
password_input.send_keys(Keys.ENTER)
time.sleep(5)

# 검색 페이지 진입
driver.get(search_url)
time.sleep(5)

# 수집 데이터 리스트
all_data = []

# 데이터 추출 함수
def extract_tweet_data(tweet):
    try:
        content = tweet.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
    except:
        content = ""

    try:
        author = tweet.find_element(By.XPATH, './/div[@data-testid="User-Name"]//span').text
    except:
        author = ""

    try:
        timestamp = tweet.find_element(By.XPATH, './/time').get_attribute('datetime')
    except:
        timestamp = ""

    return timestamp, author, content

# 스크롤 & 크롤링 반복 (50회 기준)
for _ in range(50):
    tweets = driver.find_elements(By.XPATH, '//*[@data-testid="tweet"]')
    
    for tweet in tweets:
        data = extract_tweet_data(tweet)
        if data not in all_data:
            all_data.append(data)

    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(5, 6))  # 5~6초 랜덤 대기

    # "Something went wrong" 대응 (있을 때만 새로고침)
    try:
        error_msg = driver.find_element(By.XPATH, '//*[contains(text(), "Something went wrong")]')
        if error_msg:
            print("Something went wrong detected, refreshing...")
            driver.refresh()
            time.sleep(5)
    except:
        pass

# 수집 완료 → 데이터프레임 변환
df = pd.DataFrame(all_data, columns=['timestamp', 'author', 'content'])

# 날짜 변환 및 정렬 (최신순)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.drop_duplicates(subset=['timestamp', 'author', 'content'])
df = df.sort_values('timestamp', ascending=False)

# CSV 저장
df.to_csv('x_crawl_result.csv', index=False, encoding='utf-8-sig')

print("크롤링 완료! 저장 파일: x_crawl_result.csv")

# 브라우저 종료
driver.quit()

크롤링 완료! 저장 파일: twitter_crawl_result.csv


In [50]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv('twitter_crawl_result.csv')

# 타임스탬프 datetime 변환
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 필요 컬럼 확인
print(df.head())

                  timestamp              author  \
0 2025-03-05 23:40:26+00:00           Bpay News   
1 2025-03-05 23:22:33+00:00  Bull & Bear Report   
2 2025-03-05 21:38:48+00:00     Sante Law Group   
3 2025-03-05 21:11:17+00:00    Doge Terminal AI   
4 2025-03-05 20:20:33+00:00             TknWire   

                                             content  
0  The SEC Crypto Task Force has met with represe...  
1  The US SEC and CFTC are joining forces to tack...  
2   Crypto & Government: Trump's proposal for a U...  
3  Trump's Strategic Crypto Reserve proposal aims...  
4  14 /\n\n- \n@CFTC\n   (3)* - Announces joint c...  
